In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from numba import jit
import potentials
from ewald_numpy import *
from distances import *
from scipy.special import erfc
import line_profiler

In [ ]:
def nacl_ewald(r_cutoff, n_max, k_cutoff, max_a, step_a, E=None):
    coord =np.asarray([[0,0,0], [1/2,1/2,0], [1/2,0,1/2], [0,1/2,1/2],
                           [1/2,1/2,1/2], [1/2,0,0], [0,1/2,0],[0,0,1/2]])
    if E==None:
        charges = np.asarray([-1, -1., -1, -1, 1, 1, 1, 1])
        E = Ewald(charges, (0,1), 0.1, r_cutoff, n_max, k_cutoff)
    a_array = np.arange(0.1, max_a, step_a)
    pot_lr, pot_sr, pot_self = np.zeros_like(a_array), np.zeros_like(a_array), np.zeros_like(a_array)
    for i in range(len(a_array)):
        E.params = (a_array[i], r_cutoff, n_max, k_cutoff)
        pot_lr[i] = E.pot_lr(coord)
        pot_sr[i] = E.pot_sr(coord)
        pot_self[i] = E.pot_self()
    pot_total = pot_lr + pot_sr - pot_self
    return np.asarray([a_array, pot_lr, pot_sr, pot_self, pot_total])

In [ ]:
a_array, pot_lr, pot_sr, pot_self, pot_total=nacl_ewald(10, 5, 10, 5,0.1)
print("Madelung's constant from Ewald:",np.sum(pot_total[12:]/len(pot_total[12:])/8))
print("Madelung's constant from liter:",-1.7475645946331821)

In [ ]:
charges = np.asarray([-1, -1., -1, -1, 1, 1, 1, 1])
coord = np.asarray([[0,0,0], [1/2,1/2,0], [1/2,0,1/2], [0,1/2,1/2],
                           [1/2,1/2,1/2], [1/2,0,0], [0,1/2,0],[0,0,1/2]])

E = Ewald(charges, (0,1), 0.1, 0.5, 1, 1)

fig, ax = plt.subplots(1,2,figsize=(10,5))
a_array, pot_lr, pot_sr, pot_self, pot_total =nacl_ewald(10, 10, 10, 30,1)
plt1,=ax[0].plot(a_array, pot_lr, label="$\phi_{LR}$")
plt2,=ax[0].plot(a_array, pot_sr, label="$\phi_{SR}$")
plt3,=ax[0].plot(a_array, -pot_self, label="$\phi_{self}$")
plt4,=ax[0].plot(a_array, pot_total, label="$\phi_{tot}$")

circle = plt.Circle([0.5,0.5], radius=0.5, fill=False)
box = patches.Rectangle([0,0],1,1, fill=False,linestyle='--')

def update(r=2., n=1, k=1):
    a_array, pot_lr, pot_sr, pot_self, pot_total=nacl_ewald(r, n, k, 30, 1, E)
    plt1.set_ydata(pot_lr)
    plt2.set_ydata(pot_sr)
    plt4.set_ydata(pot_total)
    fig.canvas.draw()
    
    ax[1].clear()
    circle.radius = r
    ax[1].add_patch(circle)
    ax[1].add_patch(box)
    n_array = np.mgrid[-n:n+1,-n:n+1].reshape(2,-1).T
    na_coord = np.asarray([[0,1/2],[1/2,0],[1,1/2],[1/2,1]])
    na_coord = (n_array[:,None,None] + na_coord).reshape(-1,2)
    cl_coord = np.asarray([[0,0],[0,1],[1,0],[1,1],[1/2,1/2]])
    cl_coord = (n_array[:,None,None] + cl_coord).reshape(-1,2)
    ax[1].scatter(na_coord[:,0],na_coord[:,1],s=20*min(1/n,1/r))
    ax[1].scatter(cl_coord[:,0],cl_coord[:,1],s=20*min(1/n,1/r))
    fig.canvas.draw()

ax[0].set_xlabel(r'$\alpha$')
ax[0].set_title(r'Ewald potentials V splitting param. $\alpha$')
ax[0].legend()
ax[1].set_aspect('equal')
w=interact(update,
        r=widgets.FloatSlider(min=0.5,max=10,step=0.25,value=0.5,continuous_update=False),
        k=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False),
        n=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False));

In [ ]:
E.params = 1, 2, 1,10
charges = np.asarray([-1, -1., -1, -1, 1, 1, 1, 1])
coord = np.asarray([[0,0,0], [1/2,1/2,0], [1/2,0,1/2], [0,1/2,1/2],
                           [1/2,1/2,1/2], [1/2,0,0], [0,1/2,0],[0,0,1/2]])


def update(r=0.5, k=1, n=1, alpha=0.1):
    ax.clear()
    E.params = alpha, r, n, k
    force = E.force(coord)
    na_force = np.asarray([force[5],force[6]])
    cl_force = np.asarray([force[0],force[1],force[0]])
    ax.set_xlim((-1,2))
    ax.set_ylim((-1,2))
    n_array = np.mgrid[-n:n+1,-n:n+1].reshape(2,-1).T
    na_coord = np.asarray([[1/2, 0],[0,1/2]])
    cl_coord = np.asarray([[0, 0],[1/2, 1/2],[1,0]])
    na_coord = (n_array[:,None,None] + na_coord).reshape(-1,2)
    cl_coord = (n_array[:,None,None] + cl_coord).reshape(-1,2)
    na_force = (np.zeros_like(n_array)[:,None,None] + na_force[:,:2]).reshape(-1,2)
    cl_force = (np.zeros_like(n_array)[:,None,None] + cl_force[:,:2]).reshape(-1,2)
    ax.quiver(na_coord[:,0],na_coord[:,1],na_force[:,0],na_force[:,1],angles='xy')
    ax.quiver(cl_coord[:,0],cl_coord[:,1],cl_force[:,0],cl_force[:,1],angles='xy')
    ax.scatter(na_coord[:,0],na_coord[:,1])
    ax.scatter(cl_coord[:,0],cl_coord[:,1])
    fig.canvas.draw()


fig, ax = plt.subplots()
plt.xlim((-1.5,2))
plt.ylim((-1.5,2))
ax.set_aspect("equal")

w=interact(update,
        r=widgets.FloatSlider(min=1,max=10,step=0.25,value=0.5,continuous_update=False),
        k=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False),
        n=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False),
        alpha=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=0.1, continuous_update=False));

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

charges = np.asarray([-1, -1., -1, -1, 1, 1, 1, 1])
coord = np.asarray([[0,0,0], [1/2,1/2,0], [1/2,0,1/2], [0,1/2,1/2],[1/2,1/2,1/2], [1/2,0,0], [0,1/2,0],[0,0,1/2]])
E = Ewald(charges, (0,1), 5., 10, 2, 8)

def update(r=0.5, k=1, n=1, alpha=0.1):
    ax.clear()
    E.params = alpha, r, n, k
    force = E.force(coord)
    n_array = np.mgrid[0:2,0:2,0:2].reshape(3,-1).T
    cl_coord = coord[0:4]
    na_coord = coord[4:]
    na_coord = (n_array[:,None,None] +na_coord).reshape(-1,3)
    cl_coord = (n_array[:,None,None] +cl_coord).reshape(-1,3)
    force = E.force(coord)
    cl_force = force[0:4]
    na_force = force[4:]
    na_force = (np.zeros_like(n_array)[:,None,None]+na_force).reshape(-1,3)
    cl_force = (np.zeros_like(n_array)[:,None,None]+cl_force).reshape(-1,3)
    na_force = np.asarray([f for i,f in enumerate(na_force) if (na_coord[i]<=1).all()])
    cl_force = np.asarray([f for i,f in enumerate(cl_force) if (cl_coord[i]<=1).all()])
    na_coord = np.asarray([c for c in na_coord if (c<=1).all()])
    cl_coord = np.asarray([c for c in cl_coord if (c<=1).all()])
    ax.scatter(na_coord[:,0],na_coord[:,1],na_coord[:,2])
    ax.scatter(cl_coord[:,0],cl_coord[:,1],cl_coord[:,2])
    ax.quiver(na_coord[:,0],na_coord[:,1],na_coord[:,2],
          na_force[:,0],na_force[:,1],na_force[:,2],length=0.1, normalize=True)
    ax.quiver(cl_coord[:,0],cl_coord[:,1],cl_coord[:,2]
         ,cl_force[:,0],cl_force[:,1],cl_force[:,2],length=0.1, normalize=True)
    fig.canvas.draw()


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()
w=interact(update,
        r=widgets.FloatSlider(min=1,max=10,step=0.25,value=0.5,continuous_update=False),
        k=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False),
        n=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False),
        alpha=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=0.1, continuous_update=False));